# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Unleashing the Power of AI: Exploring DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI, a leading platform that offers cutting-edge courses in deep learning. From neural networks to computer vision, expand your knowledge and skills with industry experts. Whether you're a beginner or an experienced professional, there's something here for everyone. Join a dynamic community, get hands-on experience, and unlock endless possibilities in the rapidly evolving field of AI. Don't miss this opportunity to stay ahead of the curve and redefine the future with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of AI: Exploring DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI, a leading platform that offers cutting-edge courses in deep learning. From neural networks to computer vision, expand your knowledge and skills with industry experts. Whether you're a beginner or an experienced professional, there's something here for everyone. Join a dynamic community, get hands-on experience, and unlock endless possibilities in the rapidly evolving field of AI. Don't miss this opportunity to stay ahead of the curve and redefine the future with DeepLearning.AI.

--------------------------------------------------------------------------------
Critic (to Writer):

This blogpost is eng

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of AI: Exploring DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI, a leading platform that offers cutting-edge courses in deep learning. From neural networks to computer vision, expand your knowledge and skills with industry experts. Whether you're a beginner or an experienced professional, there's something here for everyone. Join a dynamic community, get hands-on experience, and unlock endless possibilities in the rapidly evolving field of AI. Don't miss this opportunity to stay ahead of the curve and redefine the future with DeepLearning.AI.

--------------------------------------------------------------------------------

****************************************


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Based on the aggregated feedback from the SEO reviewers, it is clear that incorporating relevant keywords related to AI and deep learning in the title and content, adding internal links, and including a clear call-to-action are key aspects that could significantly enhance the piece.

Final suggestions:
1. **Optimize for SEO**: Incorporate keywords like "artificial intelligence courses," "deep learning training," or "AI industry experts" strategically in the title and throughout the content to improve search visibility.
   
2. **Enhance User Experience**: Add internal links to relevant pages within the website to improve the site's structure and provide users with easy access to more related content.

3. **Encourage Engagement**: Include a clear call-to-action at the end to prompt users to take the next step, such as "Enroll now to start your AI journey with DeepLearning.AI."

B

## Get the summary

In [15]:
print(res.summary)

Title: "Master AI Skills with DeepLearning.AI's Cutting-Edge Courses"

Immerse yourself in the world of artificial intelligence and deep learning with DeepLearning.AI's top-tier courses. From neural networks to computer vision, explore a range of topics led by industry experts. Whether you're new to AI or a seasoned professional, there's tailored content for all levels. Join a vibrant community, gain practical experience, and step into the forefront of AI innovation. Optimize your future today with DeepLearning.AI.

Don't wait, enroll now to embark on your AI journey with DeepLearning.AI.
